# Learning from Images, WS19/20
### Mahan Ghashghaie, Florian Becker

### Was ist zu erledigen?


1.   Daten laden
2.   Daten augmentieren/transformieren (on-the-fly)
3.   Bestehendes ANN als Baseline verwenden
4.   Eigenen Ansatz verwenden (z.B. mit den extrahierten Features als Input-Parameter)
5.   Evaluation
6.   Web-API zur Verfügung stellen (mittels Flask oder Django)

## Imports

In [0]:
# PyTorch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms # Transformations/Augmentations
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data # for DataLoader
from torch.autograd import Variable
from torchvision import models # pre-trained models

# 
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns

# NumPy
import numpy as np

# pandas
import pandas as pd

# utilities
import csv
import os
import copy

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

## Access GitLab

In [0]:
# this works now, I just had to set the repository to public
# otherwise https://techsupportallbugs.wordpress.com/2018/06/05/using-git-with-colab-via-ssh/
!git clone -b develop https://gitlab.beuth-hochschule.de/s76343/lfi-ws1920-project

## Data Preparation

In [0]:
# Rename filenames to class_i.JPG
# write names and labels to a CSV

csv_headers = ['ID', 'image_name', 'label']
image_csv_name = 'arc_pictures.csv' 
root_dir = '/content/lfi-ws1920-project/data/'

with open(image_csv_name, 'wt') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(csv_headers) # write header
    dirs = ['arcDataset', 'washington_image_db'] # for now both datasets will be mixed and shuffled
    
    i = 0

    for dir in dirs:
        print('collecting images for directory', dir)

        for style in os.listdir(root_dir + dir):

            print('collecting files for style ', style)
            style_folder = os.path.join(root_dir + dir, style)

            for img in os.listdir(style_folder):
                #dst = style + str(i) + ".jpg"
                src =style_folder + '/' + img 
                #dst =style_folder + '/' + dst 
                    
                # rename() function will rename all the files 
                #os.rename(src, dst)
                csv_writer.writerow([i, src, style])
                #csv_writer.writerow([i, dst, style])
                i += 1

        print(str(i), 'files found')

In [0]:
# import images from the prepared csv into a DataFrame
images_csv = pd.read_csv(image_csv_name)
images_csv

In [0]:
# show unique classes
sorted(images_csv['label'].unique())

In [0]:
# get aggregations and statistics per class
images_csv[['label', 'ID']].groupby(['label']).agg(['count'])

In [0]:
# aggregate some styles
images_csv['label'].replace(['roman'], 'ancient', inplace=True) # do the renaming on the actual frame, not a copy
images_csv['label'].replace(['postmodern'], 'modern', inplace=True)
images_csv['label'].replace(['international'], 'modern', inplace=True)
images_csv['label'].replace(['neo'], 'classicism', inplace=True)

sorted(images_csv['label'].unique())

In [0]:
# sort out 'classical', 'colonial', 'bauhaus', 'victorian', 'ancient'
images_csv = images_csv[~images_csv['label'].isin(['classical', 'colonial', 'bauhaus', 'victorian', 'ancient'])]
label_list = sorted(images_csv['label'].unique())
label_list

In [0]:
# remaining number of images
num_images = len(images_csv)
print('number of images:', num_images)

In [0]:
# do a rough 80:20 split train/validation
# shuffle the dataframe before assignment
images_csv = images_csv.sample(frac=1) # drop=True prevents .reset_index from creating a column containing the old index entries
train_upper_bound = int(num_images * .8)
images_csv_train = images_csv[0:train_upper_bound] # training
images_csv_val = images_csv[train_upper_bound:] # validation
print(len(images_csv_train), len(images_csv_val))

In [0]:
# get aggregations and statistics per class
images_csv[['label', 'ID']].groupby(['label']).agg(['count'])

In [0]:
# calculate weights per class to overcome bias due to imbalancing
weights = [2202/366, 2202/447, 2202/370, 2202/257, 2202/191, 2202/443, 2202/456, 2202/58, 2202/165]
class_weights = torch.FloatTensor(weights).cuda()
class_weights

In [0]:
# https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
# prepare labels for training and validation
partition = {'train':images_csv_train['ID'].tolist(), 'val':images_csv_val['ID'].tolist()}

train_labels = pd.Series(images_csv_train.label.values, index=images_csv_train.ID).to_dict()
val_labels = pd.Series(images_csv_val.label.values, index=images_csv_val.ID).to_dict()
labels = dict(train_labels)
labels.update(val_labels)

label_encoded = dict()
label_decoded = dict()
for i, label in enumerate(label_list):
    label_encoded[i] = label
    label_decoded[label] = i

# manually add class 'other'
#label_encoded[i+1] = 'other'
#label_decoded['other'] = i+1

print(label_decoded)
print(label_encoded)

num_classes = len(label_decoded)
print('number of classes:', num_classes)

### Dataset

In [0]:
'''
    Define DataLoaders HERE
    https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning
    https://gist.github.com/kevinzakka/d33bf8d6c7f06a9d8c76d97a7879f5cb
'''

class ArchNetDataset(torch.utils.data.Dataset):

    def __init__(self, list_ids, labels, transform = None):
        """
        Args:
            data_dir (string): path to image directory
        """
        self.labels = labels
        self.list_ids = list_ids
        self.transform = transform

    def __len__(self):
        return len(self.list_ids)

    def __getitem__(self, idx):
        '''
            This returns an actual image; will be called by the loader pipeline
            returns one sample of the data
        '''
        ID = self.list_ids[idx] # select sample

        # Load data and get label
        # es gibt bilder die grayscale sind also shape (batchsize, 1, 128, 128) und damit kommen die conv layer nicht klar
        X = Image.open(images_csv.at[ID, 'image_name']).convert('RGB')
        # Label müssen in Tensor Form ausgegeben werden (Strings --> kodieren ?)
        y = self.labels[ID]
        # label als zahl kodieren damit es später als tensor raus kommt und nicht als tuple
        # kann später wieder zurückgeführt werden über das andere dict
        y = label_decoded[y]
        if self.transform:
            X = self.transform(X)

        return (X, y)


## NN Models

In [0]:
'''
    Define NN HERE

    input: 3*224*224 image
    output: class probabilities (num_classes)

    output per layer: (in_size - kernel_size + 2*(padding) / stride) + 1
'''
# ArchNet Neural Network
class ArchNet(nn.Module):

    def __init__(self, num_channels = 3, num_classes = num_classes, print_shapes = False):
        super(ArchNet, self).__init__()
        self.kernel_size = 5
        self.stride = 1
        self.padding = 1
        self.print_shapes = print_shapes

        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, 96, 7, self.stride, self.padding),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 2, self.padding),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(3))
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 2, self.padding),
            nn.ReLU(inplace=True),
            #nn.MaxPool2d(3)
            )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 2, self.padding),
            nn.ReLU(inplace=True),
            #nn.MaxPool2d(3)
            )
        self.norm1 = nn.BatchNorm2d(384)
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(384 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 1000)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        ## Expected Dimension Error kommt hier weil der Input geflattened wird, die Conv-Layer aber ein Bild, 4 Dimensionen erwarten
        ## also die shape die auch unten bei local_batch ausgegeben wird.
        #print('input shape', x.shape)

        # Layer 1
        x = self.conv1(x)
        if self.print_shapes == True:
            print('after layer1', x.shape)

        # Layer 2
        x = self.conv2(x)
        if self.print_shapes == True:
            print('after layer2', x.shape)

        # Layer 3
        x = self.conv3(x)
        if self.print_shapes == True:
            print('after layer3', x.shape)

        x = self.conv4(x)
        if self.print_shapes == True:
            print('after layer3.5', x.shape)

        # Layer 4
        x = x.reshape(x.size(0), -1)
        x = self.drop_out(x)
        if self.print_shapes == True:
            print('after drop_out', x.shape)

        # Layer 5
        #x = x.view(x.size(0), -1)
        x = self.fc1(x)
        if self.print_shapes == True:
            print('after fc1', x.shape)


        # Layer 6
        x = self.fc2(x)
        if self.print_shapes == True:
            print('after fc2', x.shape)

        # linear layer anpassen an den geflatteten tensor
        #x = x.view(x.size(0), -1)
        # Classifier
        x = self.classifier(x)

        return x
'''
class ArchNetV2(nn.Module):
    def __init__(self, num_channels = 3, num_classes = num_classes, print_shapes = False):
        super(ArchNetV2, self).__init__()
        self.print_shapes = print_shapes

        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, 96, 7, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(96)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(256)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 5, stride=1),
            nn.ReLU(inplace=True)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride=1),
            nn.ReLU(inplace=True)
        )
        self.fc1 = nn.Linear(384 * 2 * 2, 1000)
        self.drop_out = nn.Dropout()
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x):

        # Layer 1
        x = self.conv1(x)
        if self.print_shapes == True:
            print('after layer1', x.shape)

        # Layer 2
        x = self.conv2(x)
        if self.print_shapes == True:
            print('after layer2', x.shape)

        # Layer 3
        x = self.conv3(x)
        if self.print_shapes == True:
            print('after layer3', x.shape)

        # Layer 4
        x = self.conv4(x)
        if self.print_shapes == True:
            print('after layer4', x.shape)

        # Layer 5
        x = self.fc1(x)
        if self.print_shapes == True:
            print('after layer5', x.shape)

        # Layer 6
        x = x.reshape(x.size(0), -1)
        x = self.drop_out(x)
        if self.print_shapes == True:
            print('after layer6', x.shape)

        # linear layer anpassen an den geflatteten tensor
        #x = x.view(x.size(0), -1)
        # Classifier
        x = self.fc2(x)

        return x
'''        

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
'''
    Setup run parameters here & initialise NN
'''
model = 'resnet' # TODO

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()

# Parameters
batch_size = 16 # how many samples per batch to load

# each worker works on its own batch; should be decreased (i.e. set to 0) if system resources are running out
num_workers = 1 if use_cuda else 2
pin_memory = True if use_cuda else False # whether to copy tensors into CUDA pinned memory
n_epochs_train = 50
n_epocs_val = 25

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

if model == 'archnet':
    model = ArchNet(num_classes=num_classes, num_channels=3, print_shapes=False)
    #model = ArchNetV2(num_classes=num_classes, num_channels=3, print_shapes=True)
    set_parameter_requires_grad(model, False)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, num_classes)
    #This will normalize the image in the range [-1,1]. 
    #For example, the minimum value 0 will be converted to (0-0.5)/0.5=-1, the maximum value of 1 will be converted to (1-0.5)/0.5=1.    
    #normalize = transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    input_size = 224 # TODO test

elif model == 'resnet':
    # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
    model = models.resnet50(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # expected Normalization values for ResNet
    input_size = 224 # expected value for ResNet
    n_epochs_train = 25
    n_epochs_val = 25

elif model == 'vgg':
    model = models.vgg16(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # expected Normalization values for ResNet
    input_size = 224 # expected value for ResNet
    n_epochs_train = 50
    n_epochs_val = 25

elif model == "densenet":
    model = models.densenet121(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, num_classes)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    input_size = 224
    n_epochs_train = 25

elif model == 'googlenet':
    model = models.googlenet(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    input_size = 224
    n_epochs_train = 25

elif model == 'squeezenet':
    model = models.squeezenet1_0(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
    model.num_classes = num_classes
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    input_size = 224
    n_epochs_train = 50
    n_epochs_val = 25

elif model == 'mobilenet':
    model = models.mobilenet_v2(pretrained=True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    input_size = 224
    n_epochs_train = 25
    n_epochs_val = 25

device = torch.device("cuda:0" if use_cuda else "cpu")
model = model.to(device) # send model to either GPU or CPU

#if feature_extract == True:
#    for param in model.parameters():
#        param.requires_grad = False

params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss(weight=class_weights, reduction='mean')
if use_cuda == True:
    model.cuda() # send model to the GPU
    criterion.cuda() # send optimizer to GPU

# for debugging
#torch.autograd.set_detect_anomaly(True)

In [0]:
print(model)

In [0]:
'''
    Define Tranformations here
'''

train_transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.Resize((input_size, input_size)), # Resize the image to 256x256
    #transforms.RandomResizedCrop(input_size), # Crop the image to H×W pixels about the center.
    transforms.RandomHorizontalFlip(), # flip image horizontally
    transforms.RandomRotation((-15, 15)), # apply random rotation of +-10°
    transforms.RandomPerspective(),
    transforms.RandomCrop(input_size),
    #transforms.RandomAffine((-5, 5)),
    #transforms.RandomVerticalFlip(),
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    normalize
])
print(train_transform)

valid_transform = transforms.Compose([
    transforms.Resize((input_size, input_size)), # Resize the image to 256x256
    #transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(input_size),
    #transforms.Grayscale(num_output_channels=1),                                  
    transforms.ToTensor(),
    normalize                             
])
print(valid_transform)

In [0]:
# Generators
batch_size = 1 # TODO
training_set = ArchNetDataset(partition['train'], labels, transform=train_transform)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=4, num_workers=num_workers, shuffle=True, pin_memory=pin_memory)

valid_set = ArchNetDataset(partition['val'], labels, transform=valid_transform)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=pin_memory)

In [0]:
'''
    Let’s visualize a few training images so as to understand the data augmentations.
'''

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize = (25,3))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(training_generator))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[label_encoded[int(x)] for x in classes])


## Training

In [0]:
%%time

model.train() # set to TRAIN mode

for epoch in range(n_epochs_train):
    
    print('Epoch {}/{}'.format(epoch, n_epochs_train - 1))
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0
    
    for local_batch, local_labels in training_generator:
        # Transfer to GPU
        local_batch = Variable(local_batch)
        #print(local_labels)
        local_labels = Variable(local_labels)
        
        # zero parameter gradients
        optimizer.zero_grad()

        if use_cuda == True:
            local_batch = local_batch.cuda()
            local_labels = local_labels.cuda()

        # Forward + Backward + Optimize
        outputs = model(local_batch)
        loss = criterion(outputs, local_labels)

        _, preds = torch.max(outputs, 1)

        #print('Epoch {}, train Loss: {:.3f}'.format(epoch, loss.item()))
        loss.backward() # computes the gradients
        optimizer.step() # adapt the weights

        # statistics
        running_loss += loss.item() * local_batch.size(0)
        running_corrects += torch.sum(preds == local_labels.data)

    epoch_loss = running_loss / len(training_generator.dataset)
    epoch_acc = running_corrects.double() / len(training_generator.dataset)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('TRAIN', epoch_loss, epoch_acc))


## VALIDATION

In [0]:
%%time

model.eval() # set to VALIDATION mode

val_acc_history = []

#best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
n_epochs = 50

for epoch in range(n_epochs_val):
    
    print('Epoch {}/{}'.format(epoch, n_epochs_val - 1))
    print('-' * 10)

    running_loss = 0.0
    running_corrects = 0
    
    for local_batch, local_labels in valid_loader:
        # Transfer to GPU
        local_batch = Variable(local_batch)
        #print(local_labels)
        local_labels = Variable(local_labels)
        
        # zero parameter gradients
        #optimizer.zero_grad()

        if use_cuda == True:
            local_batch = local_batch.cuda()
            local_labels = local_labels.cuda()

        # Forward + Backward + Optimize
        outputs = model(local_batch)
        loss = criterion(outputs, local_labels)

        _, preds = torch.max(outputs, 1)

        # statistics
        running_loss += loss.item() * local_batch.size(0)
        running_corrects += torch.sum(preds == local_labels.data)

    epoch_loss = running_loss / len(valid_loader.dataset)
    epoch_acc = running_corrects.double() / len(valid_loader.dataset)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('VALIDATION', epoch_loss, epoch_acc))

    # deep copy the model
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        #best_model_wts = copy.deepcopy(model.state_dict())
    
    val_acc_history.append(epoch_acc)

## PREDICTION/TEST

In [0]:
''' '''
# USE THIS IF LOADING A PRE-TRAINED MODEL
model = models.resnet50(pretrained=True)
#model = ArchNet(num_classes=9)
#num_ftrs = model.classifier.in_features
#model.classifier = nn.Linear(num_ftrs, num_classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model.load_state_dict(torch.load('/content/lfi-ws1920-project/models/resnet.pt'))
model.to(device)
model.eval()


In [0]:
correct = 0
wrong = 0
k = 5 # show probabilities for top k classes
# (1) load custom_images folder, which contains 10 images per label
for dir in ['custom_images']:

    # (2) iterate each style
    for style in os.listdir(root_dir + dir):

        print('test images for style ', style)
        style_folder = os.path.join(root_dir + dir, style)

        for img in os.listdir(style_folder):
            print('test image:', style_folder + '/' + img)
            img_t = Image.open(style_folder + '/' + img).convert('RGB')
            batch_t = torch.unsqueeze(train_transform(img_t), 0)

            # (3) predict the class
            out = model(batch_t.cuda())

            #print(torch.nn.functional.softmax(out) * 100)
            # CrossEntropy uses log-softmax values -> calculate actual probability values
            top5_prob, top5_label = torch.topk(torch.nn.functional.softmax(out, dim=1) * 100, k)
            print('Top 5 Probalities: ', top5_prob.tolist())
            print('Top 5 Classes: ', top5_label.tolist())
            print('------' * 5)

            # Now, we need to find out the index where the maximum score in output vector out occurs. We will use this index to find out the prediction.
            _, index = torch.max(out, 1)
            pred_class = label_encoded[int(index[0])]
            print('predicted class =', pred_class)

            # (4) test against groundtruth
            if pred_class == style:
                print('Correclty predicted!')
                correct += 1
            else:
                print('This is wrong!')
                wrong += 1
            
            print('\n')

print('Correct: ', correct)
print('Wrong: ', wrong)

# Flask Stuff comes here


## Misc

In [0]:
model_loc = "/content/archnet.pt"

In [0]:
torch.save(model.state_dict(), model_loc)

In [0]:
archnet2_point_0 = ArchNet(num_classes=14)
archnet2_point_0.load_state_dict(torch.load(model_loc))
archnet2_point_0.eval()

In [0]:
!pip install flask-ngrok


## Server

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request, Response
import cv2

In [0]:
app = Flask(__name__)
run_with_ngrok(app)

In [0]:
@app.route('/api/v1/test', methods=['POST'])
def get_tasks():
    nparr = np.fromstring(request.data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    response = {'message': 'image received.', 'size':'{}x{}'.format(img.shape[1], img.shape[0])}
    return jsonify(response)

@app.route('/api/v1/predict', methods=['POST'])
def style_prediction():
    nparr = np.frombuffer(request.data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img_resized = cv2.resize(img, (224, 224)) 
    img_resized = np.transpose(img_resized,(2,0,1))
    img_tensor = torch.from_numpy(img_resized)
    img_tensor = img_tensor[None, :, :, :]
    img_tensor = img_tensor.type(torch.FloatTensor)
    y_pred = model(img_tensor)
    _, index = torch.max(y_pred, 1)
    predicted_class = label_encoded[int(index[0])]
    response = {'message': 'image received.', 'initial_size':'{}x{}'.format(img.shape[1], img.shape[0]), "prediction": predicted_class}
    return jsonify(response)

In [0]:
app.run()

## Client


---
Den Teil müsste man sich rauskopieren und dann in einem eigenen notebook oder script ausführen --> oben beim output von app.run() die ngrok url kopieren und beim Aufruf von send_image_to_api den api_baseurl parameter setzen

Dann muss man auch ein eigenes Bild schicken



In [0]:
import requests

In [0]:
#choose any img from colab, may also use images_csv for that
content_type = 'image/jpeg'
headers = {'content-type': content_type}

In [0]:
def send_image_to_api(image_location, api_baseurl="http://127.0.0.1:5000", api_endpoint="/api/v1/predict"):
    img = open(image_location, 'rb').read()
    content_type = 'image/png'
    headers = {'content-type': content_type}
    targeturl = "{}{}".format(api_baseurl, api_endpoint)
    return requests.post(targeturl, data=img, headers=headers)

In [0]:
response = send_image_to_api(images_csv['image_name'][0], api_endpoint = "/api/v1/test")
response.text

# Visualization

## Display Neural Network layer(s)

* https://colab.research.google.com/github/Niranjankumar-c/DeepLearning-PadhAI/blob/master/DeepLearning_Materials/6_VisualizationCNN_Pytorch/CNNVisualisation.ipynb#scrollTo=qv-nJbDFuNuN
* https://software.intel.com/en-us/articles/visualising-cnn-models-using-pytorch


In [0]:
#custom function to conduct occlusion experiments

def occlusion(model, image, label, occ_size = 50, occ_stride = 50, occ_pixel = 0.5):
  
    #get the width and height of the image
    width, height = image.shape[-2], image.shape[-1]
  
    #setting the output image width and height
    output_height = int(np.ceil((height-occ_size)/occ_stride))
    output_width = int(np.ceil((width-occ_size)/occ_stride))
  
    #create a white image of sizes we defined
    heatmap = torch.zeros((output_height, output_width))
    
    #iterate all the pixels in each column
    for h in range(0, height):
        for w in range(0, width):
            
            h_start = h*occ_stride
            w_start = w*occ_stride
            h_end = min(height, h_start + occ_size)
            w_end = min(width, w_start + occ_size)
            
            if (w_end) >= width or (h_end) >= height:
                continue
            
            input_image = image.clone().detach()
            
            #replacing all the pixel information in the image with occ_pixel(grey) in the specified location
            input_image[:, :, w_start:w_end, h_start:h_end] = occ_pixel
            
            #run inference on modified image
            output = model(input_image.cuda())
            output = nn.functional.softmax(output, dim=1)
            prob = output.tolist()[0][label]
            
            #setting the heatmap location to probability value
            heatmap[h, w] = prob 

    return heatmap

In [0]:
#images,classes = next(iter(training_generator))
#run the model on the images
outputs = model(inputs.cuda())

#get the maximum class 
labels, pred = torch.max(outputs.data, 1)

#running inference on the images without occlusion

#vgg16 pretrained model

outputs = model(inputs.cuda())
print(outputs.shape)

#passing the outputs through softmax to interpret them as probability
outputs = nn.functional.softmax(outputs, dim = 1)

#getting the maximum predicted label
prob_no_occ, pred = torch.max(outputs.data, 1)

#get the first item
prob_no_occ = prob_no_occ[0].item()

print(prob_no_occ)

In [0]:

heatmap = occlusion(model, inputs, pred[0].item(), 32, 14)

In [0]:
imgplot = sns.heatmap(heatmap, xticklabels=False, yticklabels=False, vmax=prob_no_occ)
figure = imgplot.get_figure()    
#figure.savefig('svm_conf.png', dpi=400)

In [0]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[label_encoded[int(x)] for x in classes])